In [1]:
import pandas as pd
import tiktoken
# from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv,dotenv_values,find_dotenv
import json
from tenacity import retry, wait_random_exponential, stop_after_attempt

c:\Users\belifakb\Downloads\UPM\GEN-AI\Development\AZURE-AI-VECTOR-SEARCH\openaiev\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# FILE_PATH = "C:/Users/belifakb/Downloads/UPM/GEN-AI/Development/AZURE-AI-VECTOR-SEARCH/azure_ai_vector_search/doc/CHAP04-BIOLOGY-CLASS11.pdf"
# FILE_PATH = "C:/Users/belifakb/Downloads/UPM/GEN-AI/Development/AZURE-AI-VECTOR-SEARCH/azure_ai_vector_search/doc/Azure Common Data Platform - Operation Manual - CDP.pdf"
FILE_PATH = "C:/Users/belifakb/Downloads/UPM/GEN-AI/Development/AZURE-AI-VECTOR-SEARCH/azure_ai_vector_search/doc/incidents_2023_pulp.xlsx"

In [3]:
# Divide the text into chunks of chunk_length 
# [ default is 500] characters
def get_chunks(fulltext:str,chunk_length =500) -> list:
    text = fulltext

    chunks = []
    while len(text) > chunk_length:
        last_period_index = text[:chunk_length].rfind('.')
        if last_period_index == -1:
            last_period_index = chunk_length
        chunks.append(text[:last_period_index])
        text = text[last_period_index+1:]
    chunks.append(text)

    return chunks

In [4]:
filename = FILE_PATH

In [5]:
FILE_PATH.split('/')[-1]

'incidents_2023_pulp.xlsx'

In [6]:
# full_doc_text = get_pdf_data(filename)

In [7]:
# print(f'Full doc text length: {len(full_doc_text)}')

In [8]:
# Lines =get_chunks(full_doc_text,500)

In [9]:
# len(Lines)

In [10]:
# type(Lines)

In [11]:
from openai import AzureOpenAI
import os

In [12]:
load_dotenv(find_dotenv("../.env.sample"),override = True)

True

In [13]:
client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_KEY"), 
  api_version = "2023-05-15",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

In [14]:
model: str = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_ID")
# MODEL_NAME: str = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_ID")

In [15]:
# Function to generate embeddings for title and content fields, also used for query embeddings
@retry(wait=wait_random_exponential(min=1, max=5), stop=stop_after_attempt(2))
def generate_embeddings(text, model=model):
    return client.embeddings.create(input = [text], model=model).data[0].embedding

def get_embedded_tokenSize(text):
    encoding = tiktoken.encoding_for_model("text-embedding-ada-002")
    return int(len(encoding.encode(text)))

In [16]:
input_data = []

In [17]:
def get_excel_data(file_path):
    df = pd.read_excel(file_path)
    return df

In [18]:
# import time
# dataframe = get_excel_data(filename)
# # dataframe = dataframe.dropna()
# items = []
# for index,row in dataframe.iterrows():
#     # print(index)
#     # print(row['Number'])
#     # time.sleep(1)
#     row_dict = row.to_dict()
#     # for key, value in row_dict.items():
#     result= ', '.join([f"{str(key)} : {str(value)}" for key, value in row_dict.items()])


# # Convert the list to a string
# # result = '\n'.join(items)
# # print(result)
#     print(result)

In [19]:
import time
dataframe = get_excel_data(filename)
# dataframe = dataframe.dropna()

input_data = []


# %%time
try:
    for index,row in dataframe.iterrows():
        # print(index)
        # print(row['Number'])
        # time.sleep(1)
        row_as_string = ', '.join([str(item) for item in row])
        # print(row_as_string)
        if index != 200 and get_embedded_tokenSize(row_as_string) < 8607:
            input_data.append({
                "@search.action":"upload",
                "id"            : str(index),
                "line"          : str(row_as_string),
                "filename"      : FILE_PATH.split('/')[-1],
                "embedding"     : generate_embeddings(row_as_string) #model.encode(row,show_progress_bar=True)[0].tolist()
            })
        else:
            print(f"SKIPPING {row['Number']} at Index Number {index} with Tokenizer Size {get_embedded_tokenSize(row_as_string)} ")
except Exception as e:
    print(index)
    print(row['Number'])
    print(e)


# Output embeddings to docVectors.json file
with open("../output/ExcelAzureDoclineVectorspulp.json", "w") as f:
    json.dump(input_data, f)

In [20]:
# %%time
# for line in Lines:
#     d = {}
#     d['id'] = str(counter)
#     d['line'] = line
#     d['embedding'] = generate_embeddings(line)
#     d['filename'] = FILE_PATH.split('/')[-1]
#     counter = counter + 1
#     input_data.append(d)

In [21]:
input_data[0]

{'@search.action': 'upload',
 'id': '0',
 'line': 'SAP Pulp ERP Logistics, PULP, 02.01.2023 19.29.44, 03.01.2023 17.15.59, INC2555804, post good issue - P11 - Pulp Loading Cockpit, Hi, I am having problem when I try to post good issue - P11 - Pulp Loading Cockpit. \nDelivery number 55090064. \nError in enclosed file. \nMany thanks\nJustyna, Other, 3 - Moderate, Closed, HCL Service Desk, Vikram Bisht, Goods issue has been posted now as checked with Business user., Solved (Permanently)',
 'filename': 'incidents_2023_pulp.xlsx',
 'embedding': [0.007082273252308369,
  -0.003035514382645488,
  -0.012206150218844414,
  -0.014193031936883926,
  -0.02996702492237091,
  0.017589960247278214,
  -0.019669421017169952,
  -0.010034110397100449,
  -0.027844835072755814,
  -0.012626315467059612,
  0.02022489346563816,
  0.010205024853348732,
  -0.03894003853201866,
  0.001956616062670946,
  -0.008666793815791607,
  -0.0008888448937796056,
  0.009492880664765835,
  -0.022475268691778183,
  0.018729390

In [22]:
len(input_data)

38

In [23]:
len(input_data[0]['embedding'])

1536